In [8]:
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import urllib3
import datetime
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

GAMES = ['lotto-max', 'lotto-649', 'daily-grand']

hardcode_grand = ['5/5 + GN', '5/5', '4/5 + GN', '4/5', '3/5 + GN', '3/5', '2/5 + GN', '1/5 + GN']
size_grand = [(5, True), (5, False), (4, True), (4, False), (3, True), (3, False), (2, True), (1, True)]
intersections_grand = dict(zip(size_grand, hardcode_grand))

hardcode_max = ['7/7', '7/7', '6/7 + Bonus', '6/7', '5/7', '5/7', '4/7', '4/7', '3/7 + Bonus']
size_max = [(7, False), (7, True), (6, True), (6, False), (5, False), (5, True), (4, False), (4, True), (3, True)]
intersections_max = dict(zip(size_max, hardcode_max))

hardcode_649 = ['6/6', '5/6 + Bonus', '5/6', '4/6', '3/6', '2/6 + Bonus']
size_649 = [(6, False), (5, True), (5, False), (4, False), (3, False), (2, True)]
intersections_649 = dict(zip(size_649, hardcode_649))

intersections = dict(zip(GAMES, [intersections_max, intersections_649, intersections_grand]))

def get_dates(date1, date2, game):
    
    if game == GAMES[0]:
        dates_of_game = []
        d = date1
        start_tuesdays = datetime.date(2019, 5, 11)
        while d <= date2 and d <= start_tuesdays:
            if d.weekday() == 4: # fridays
                dates_of_game.append(d)
            d += datetime.timedelta(1)

        while d <= date2 and d > start_tuesdays:
            if d.weekday() == 4 or d.weekday() == 1: # fridays and tuesdays
                dates_of_game.append(d)
            d += datetime.timedelta(1)
            
        return dates_of_game
    
    elif game == GAMES[1]:
        dates_of_game = []
        d = date1
        while d <= date2:
            if d.weekday() == 5 or d.weekday() == 2: # saturdays and wednesdays
                dates_of_game.append(d)
            d += datetime.timedelta(1)
        return dates_of_game
    
    elif game == GAMES[2]:
        dates_of_game = []
        d = date1
        while d <= date2:
            if d.weekday() == 0 or d.weekday() == 3: # mondays and thursday
                dates_of_game.append(d)
            d += datetime.timedelta(1)
        return dates_of_game
    
    else:
        return list()

def get_url(year, month, day, game):
    url_template = 'https://lottery.olg.ca/en-ca/winning-numbers/' + game + '/winning-numbers?'
    startDate = datetime.date(year, month, day) - datetime.timedelta(1)
    endDate = datetime.date(year, month, day) + datetime.timedelta(1)
    url = url_template + 'startDate=' + str(startDate) + '&' + 'endDate=' + str(endDate)
    return url

def get_data(url, game):
    data = dict()
    http = urllib3.PoolManager()
    r = http.request('GET', url)
    soup = BeautifulSoup(r.data, 'html.parser')
    
    # getting numbers
    nums = soup.findAll('span', attrs={'class': 'winning-numbers-number'})
    win_numbers = []
    for number in nums:
        win_numbers.append(number.text)
    
    # getting money wins from the table
    table = soup.find_all('tbody')[0]
    values = dict()
    
    if game != GAMES[2]:
        for row in table.find_all('tr'):
            value = float(row.find('td', {'class':'chart-prize-col'}).text[1:].replace(',',''))
            winner = row.find('td', {'class':'chart-winners-col'}).text
            values[winner] = value
    else: # for a daily grand trick
        for row in table.find_all('tr'):
            value = row.find('td', {'class':'chart-prize-col'}).text[1:].replace(',','')
            if value == '1000 a DAY for LIFE':
                value = 365000 * 30.
            elif value == '25000 a YEAR for LIFE':
                value = 25000 * 30.
            else:
                value = float(value)
            winner = row.find('td', {'class':'chart-winners-col'}).text
            values[winner] = value 
    
    # getting a date
    current_date = soup.find('p', {'class':'large-date'}).text
    
    ### saving
    if game == GAMES[0]: # lotto max then 7 numbers
        data['numbers'] = [int(a) for a in win_numbers[:7]]
        data['bonus'] = int(win_numbers[7])
        
    elif game == GAMES[1]: # lotto 6/49 then 6 numbers
        data['numbers'] = [int(a) for a in win_numbers[:6]]
        data['bonus'] = int(win_numbers[6])
    
    elif game == GAMES[2]: # daily grand then 5 numbers + 1 bonus
        data['numbers'] = [int(a) for a in win_numbers[:5]]
        data['bonus'] = int(win_numbers[5])
    
    data['date'] = current_date
    data['money'] = values
    return data

def total_sum(database, info, game):
    total = 0
    free_plays = 0
    for numbers in database:
        
        win = 0.0
        free_play = 0
        size = 0
        bonus = True
        inters = 0
        
        if game != GAMES[2]: # if not daily grand          
            inters = set(numbers).intersection(set(info['numbers']))
            size = len(inters)
            bonus = info['bonus'] in set(numbers)
                
        else: # if daily grand
            gn = numbers[-1]
            inters = set(numbers[:-1]).intersection(set(info['numbers']))
            size = len(inters)
            bonus = info['bonus'] == gn 
        
        if game == GAMES[0]:
            if (size, bonus) == (3, False):
                free_play = 1
                free_plays += 1
                
        elif game == GAMES[1]:
            if (size, bonus) == (2, False):
                free_play = 1
                free_plays += 1
                
        elif game == GAMES[2]:
            if (size, bonus) == (0, True):
                free_play = 1
                free_plays += 1
                
        if (size, bonus) in intersections[game]:
            key = intersections[game][(size, bonus)]
            win = info['money'][key]
            
        total += win
        print('You win ${:>6} and {} free play for a ticket with numbers {}'.format(win, free_play, numbers))
    print('\nYour total win on {} is ${} and {} free plays'.format(info['date'], total, free_plays))
    return np.array([total, free_plays])
    
def summary(date1, date2, game):
    
    print('Game is ' + game.upper())
    dates_of_game = get_dates(date1, date2, game)
    print()
    
    super_total = np.array([0., 0.])
    for friday in dates_of_game:
        day = friday.day
        month = friday.month
        year = friday.year
        info = get_data(get_url(year, month, day, game), game)

        print('On {} \nwinning numbers are: {} - {}'.format(info['date'], info['numbers'], info['bonus']))
        print('Table of winnings:')
        for key in info['money']:
            if int(info['money'][key]) == 365000 * 30:
                print('{:>15} ----- ${}'.format(key, '1,000 per DAY for LIFE'))
            elif int(info['money'][key]) == 25000 * 30:
                print('{:>15} ----- ${}'.format(key, '25,000 per YEAR for LIFE'))  
            else:
                print('{:>15} ----- ${}'.format(key, info['money'][key]))
        print()
        
        moneey = total_sum(database, info, game)
        print('\n\n---------------------------------------------------')
        
        super_total += moneey
    print('\n\nTOTAL = ${} and {} free plays'.format(super_total[0], super_total[1]))


def get_encore_url(year, month, day, game):
    url_template = 'https://lottery.olg.ca/en-ca/winning-numbers/' + game + '/winning-numbers?'
    startDate = datetime.date(year, month, day) - datetime.timedelta(1)
    endDate = datetime.date(year, month, day) + datetime.timedelta(1)
    url = url_template + 'startDate=' + str(startDate) + '&' + 'endDate=' + str(endDate) + '#lotto-max-game3'
    return url

# Set up the game:
# 0 - lotto-max
# 1 - lotto-6/49
# 2 - daily grand
game = GAMES[2]


# Set up the dates range:

year_from = 2019
month_from = 5
day_from = 1

year_to = 2019
month_to = 5
day_to = 21

date1 = datetime.date(year_from, month_from, day_from)
date2 = datetime.date(year_to, month_to, day_to)
print('Your tickets are: ')
filename = game + '.txt'
database = np.loadtxt(filename, dtype=np.int64)
print(database)
print('\n\n')

summary(date1, date2, game)

Your tickets are: 
[[ 3  8 17 21 28  2]
 [ 7  8 13 27 28  3]
 [ 1 17 30 31 49  5]]



Game is DAILY-GRAND

On Thu, May 2 
winning numbers are: [7, 8, 24, 38, 39] - 3
Table of winnings:
       5/5 + GN ----- $1,000 per DAY for LIFE
            5/5 ----- $25,000 per YEAR for LIFE
       4/5 + GN ----- $1000.0
            4/5 ----- $500.0
       3/5 + GN ----- $100.0
            3/5 ----- $20.0
       2/5 + GN ----- $10.0
       1/5 + GN ----- $4.0

You win $   0.0 and 0 free play for a ticket with numbers [ 3  8 17 21 28  2]
You win $  10.0 and 0 free play for a ticket with numbers [ 7  8 13 27 28  3]
You win $   0.0 and 0 free play for a ticket with numbers [ 1 17 30 31 49  5]

Your total win on Thu, May 2 is $10.0 and 0 free plays


---------------------------------------------------
On Mon, May 6 
winning numbers are: [3, 24, 29, 36, 43] - 3
Table of winnings:
       5/5 + GN ----- $1,000 per DAY for LIFE
            5/5 ----- $25,000 per YEAR for LIFE
       4/5 + GN ----- $1000.0
  